## Python Code Executor

In this challenge, we have to read the code line by line and then execute them by passing some obstacles.

In [3]:
import re
import io
from typing import Union, TextIO

In [5]:
def _to_pure_code(line: str) -> list[str]:
    """Removes comments and then strips from the right side."""
    return list(filter(lambda x: x, re.split(r'[\s\t]*#.*', line)))

In [29]:
def executor(code: Union[str, TextIO]) -> list[str]:
    """It distinguishes between single-line and multi-line commands."""
    if isinstance(code, io.TextIOBase):
        code = code.read()
    code = code.split('\n')

    # If we run a code block line by line, we will encounter an error;
    # so we have to run them after closing.
    codeblock = []
    reports = []

    for idx, line in enumerate(code):
        line = _to_pure_code(line)
        if not line:
            reports.append(f'L#{idx+1:0>2}: Nothing found here')
            continue

        singlecode = re.match(r'[^\s\t].+[^:]$', line[0])
        if singlecode is not None:
            if not codeblock:
                reports.append(f'L#{idx+1:0>2}: Just a single line code')
                exec(singlecode.group())
            else:
                reports.append(f'L#{idx+1:0>2}: The code block will be executed first')
                exec('\n'.join(codeblock))
                exec(singlecode.group())
                codeblock.clear()
            continue

        statement = re.match(r'.+:$', line[0])
        if statement is not None:
            reports.append(f'L#{idx+1:0>2}: A statement found')
            codeblock.append(statement.group())
            continue

        substatement = re.match(r'[\s\t]+.+', line[0])
        if substatement is not None:
            reports.append(f'L#{idx+1:0>2}: This is a sub statement')
            codeblock.append(substatement.group())
            continue

    return reports

In [30]:
code = """
print('Hello, world!')

n = 5  # This is an unnecessary comment
print(f'Ready for {n} repetitions...')
for _ in range(n):
    print('I <3 u!')
print(f'The loop is finished.')

print('Goodbye, dude!')
"""
code

"\nprint('Hello, world!')\n\nn = 5  # This is an unnecessary comment\nprint(f'Ready for {n} repetitions...')\nfor _ in range(n):\n    print('I <3 u!')\nprint(f'The loop is finished.')\n\nprint('Goodbye, dude!')\n"

In [31]:
reports = executor(code)

Hello, world!
Ready for 5 repetitions...
I <3 u!
I <3 u!
I <3 u!
I <3 u!
I <3 u!
The loop is finished.
Goodbye, dude!


In [32]:
for report in reports:
    print(report)

L#01: Nothing found here
L#02: Just a single line code
L#03: Nothing found here
L#04: Just a single line code
L#05: Just a single line code
L#06: A statement found
L#07: This is a sub statement
L#08: The code block will be executed first
L#09: Nothing found here
L#10: Just a single line code
L#11: Nothing found here


The worst problem of this implementation is that you can't use `#` and `\n` in a string...